<a href="https://colab.research.google.com/github/jgomezpe/primenumbers/blob/main/lifrac_c%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Defining Upper and Lower Bounding Functions of $li(x)$ with $\displaystyle O\left(\sqrt{\frac{x}{\log(x)}}\right)$ Error Using Truncated Asymptotic Series</h2>

This program checks Corollary 29 of the paper "Defining Upper and Lower Bounding Functions of $li(x)$ with $\displaystyle O\left(\sqrt{\frac{x}{\log(x)}}\right)$ Error Using Truncated Asymptotic Series" by <b>Professor Jonatan Gomez</b> from <b>Universidad Nacional de Colombia</b>.

<h3>Corollary 29</h3>
$\displaystyle \underline{li}(x) \le \pi(x)$ for all $ e \le x < 2.09*10^{9}$.

<i>Remember to run each cell one by one in the given order.</i>

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <math.h>

/**
  * Determines if tracing messagess are printed or not
  */
bool TRACE = true; // Set to false if you do not want to trace algorithm

/**
  * Determines the number of bits used for representing an integer
  */
int bits(){
  uint i=1;
  int k=0;
  while(i>0){
    i <<= 1;
    k++;
  }
  return k;
}
int BITS = bits();

/** Determines the numbers of bits to shift a number for
  * determining the position of a number's mark on the array of bits
  */
int shift(){
  int bits = BITS;
  int k=-1;
  while(bits>0){
    bits >>= 1;
    k++;
  }
  return k + 1;
}
int SHIFT = shift();

/**
  * Mask for getting the bit inside the number that represents the number
  */
long MASK = (BITS<<1)-1;

/**
  * Position in the array of the bit representing the odd number n
  */
long pos(long n){ return (n >> SHIFT); }

/**
  * Bit representing the odd number n.
  */
long bit(long n){ return (n & MASK)>>1; }

/**
  * Prime numbers computed up to N
  */
long N;

/**
  * Array with odd numbers marked as prime == 0 or not prime == 1
  */
int* PRIMES=NULL;

/**
  * Get the value (0 : is prime or 1: is not prime) of the bit representing odd number n.
  */
int get(long n){
  int p = pos(n);
  int b = bit(n);
  return (PRIMES[p] & (1<<b))>>(b);
}

/**
  * Initializes array (up to n) of values as possible primes (all bit set to 0)
  */
void init(long n){
  if(PRIMES != NULL ) delete[] PRIMES;
  int m = (int)(n>>SHIFT) + 1;
  PRIMES = new int[m];
  for(int i=0; i<m; i++) PRIMES[i] = 0;
  N = n;
}

/**
  * Mark the number as not prime (set bit to 1)
  */
void mark(long n){
  int p = pos(n);
  int b = bit(n);
  PRIMES[p] |= (1<<b);
}

/**
  * Mark as not prime all multiples of number p starting at p*p
  */
void mark_multiples(long n, long p){
  long p2 = p<<1;
  for(long i=p*p; i<=n; i+=p2) mark(i);
}

/**
  * Runs the Sieve of Eratosthenes only considering odd numbers
  */
void sieve(long n){
  init(n);
  int m = (int)sqrt(n);
  mark(1);
  if(TRACE) std::cout << "Marks multiples of:\n";
  for(long i=3; i<=m; i+=2)
    if(get(i)==0){
      if(TRACE) std::cout << i << "..";
      mark_multiples(n, i);
      if(TRACE) std::cout << "..done\n";
    }
}

/**
  * Determines if a number is prime or not
  */
bool isprime(long n){ return (n==2 || (n%2!=0 && get(n)==0)); }

/**
  * Counts the number of prime number in a given interval
  */
long primes(long start, long end){
  long count = start<=2?1:0;
  if(start % 2 == 0) start++;
  for(long i=start; i<=end; i+=2)
    if(get(i)==0) count++;
  return count;
}

/**
  * Computes $\under{li}(x)$
  */
double li_under(double x){
  double ln_x = log(x);
  int n = (int)ln_x;
  double kappa = 0.18668230890762;
  int m = (int)(kappa*ln_x);
  double alpha = kappa*ln_x - m;
  if(m==0) return alpha*x/ln_x;
  double s = 1;
  double p = 1;
  for(int k=1; k<m; k++){
    p *= k/ln_x;
    s += p;
  };
  s += alpha*p*m/ln_x;
  return s*x/ln_x;
}

/**
  * Estimates the maximum integer value ($x_{i+1}$) such that $\underline{li}(x_{i+1}) \le \pi(x_{i})$
  */
long search( long pi, long a, long b ){
  while( a < b ){
    long m = (a+b)/2;
    double l = li_under((double)m);
    if( pi < l ) b=m-1;
    else a=m+1;
  };
  if(a>b) return b;
  return a;
}

/**
  * Checks Corollary 29 up to $b$
  */
bool check_pi_li_under(long b){
  long I = 0;
  long a = 2;
  long pi = 1;
  std::cout << "$I$ & $x$ & $\\underline{li}(x) & $\\pi(x)$ \\\\ \n";
  std::cout << "0 & 2 & - & 1 \\\\ \n";
  long x = search(pi, a, b);
  while(x < b){
    pi += primes(a+1,x);
    I++;
    std::cout << I << " & " << x << " & " << li_under(x) << " & " << pi << "\\\\ \n";
    a = x;
    x = search(pi, a, b);
  };
  return (x>=b);
}

int main() {
  std::cout << "Program developed by Professor Jonatan Gomez\n";
  TRACE = 1;
  long n = 2090200000;
  std::cout << "Sieve of Eratosthenes\n";
  sieve(n+1);
  std::cout << "Checking Corollary 29\n";
  std::cout << check_pi_li_under(n);
  delete[] PRIMES;
  return 0;
}

Overwriting main.cpp


In [ ]:
%%script bash
g++ main.cpp -std=c++11 -o sieve

In [ ]:
!./sieve

Se han truncado las últimas 5000 líneas del flujo de salida.
8412 & 868288761 & 4.446e+07 & 44470683\\ 
8413 & 868507777 & 4.44707e+07 & 44481343\\ 
8414 & 868727206 & 4.44813e+07 & 44491988\\ 
8415 & 868946328 & 4.4492e+07 & 44502719\\ 
8416 & 869167224 & 4.45027e+07 & 44513377\\ 
8417 & 869386619 & 4.45134e+07 & 44524045\\ 
8418 & 869606224 & 4.4524e+07 & 44534618\\ 
8419 & 869823874 & 4.45346e+07 & 44545343\\ 
8420 & 870044657 & 4.45453e+07 & 44556099\\ 
8421 & 870266081 & 4.45561e+07 & 44566821\\ 
8422 & 870486809 & 4.45668e+07 & 44577594\\ 
8423 & 870708588 & 4.45776e+07 & 44588345\\ 
8424 & 870929917 & 4.45883e+07 & 44599110\\ 
8425 & 871151538 & 4.45991e+07 & 44609803\\ 
8426 & 871371677 & 4.46098e+07 & 44620428\\ 
8427 & 871590421 & 4.46204e+07 & 44631021\\ 
8428 & 871808509 & 4.4631e+07 & 44641562\\ 
8429 & 872025528 & 4.46416e+07 & 44652227\\ 
8430 & 872245102 & 4.46522e+07 & 44662947\\ 
8431 & 872465812 & 4.46629e+07 & 44673547\\ 
8432 & 872684054 & 4.46735e+07 & 44684084\\ 